#### Claude includes a built-in web search tool that lets it search the internet for current or specialized information to answer user questions. Unlike other tools where you need to provide the implementation, Claude handles the entire search process automatically - you just need to provide a simple schema to enable it.

#### Important note: Your organization must enable the Web Search tool in the settings console before using it. You can find this setting here: https://console.anthropic.com/settings/privacy

### Practical Usage

The web search tool works best for:

- Current events and recent developments
- Specialized information not in Claude's training data
- Fact-checking and finding authoritative sources
- Research tasks requiring up-to-date information

Simply include the schema in your tools array when making API calls, and Claude will automatically decide when a web search would help answer the user's question.

# dependencies

In [1]:
from anthropic import Anthropic

from dotenv import load_dotenv
import os   
load_dotenv()

import logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
    )
logger = logging.getLogger(__name__)

try:
    client = Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )
    logger.info("Anthropic client initialized successfully.")
except Exception as e:
    logger.error(f"Failed to initialize Anthropic client: {e}")
    raise e

2025-08-08 13:53:57 - __main__ - INFO - Anthropic client initialized successfully.


# global config

In [2]:
MODEL="claude-3-5-sonnet-20241022"
TEMPERATURE=1

# helper functions

In [3]:
def add_user_message(messages, message_content):
    if isinstance(message_content, list):
        user_message = {
            "role": "user",
            "content": message_content
        }
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": str(message_content)}]}

    messages.append(user_message)


def add_assistant_message(messages, message_content):
    if isinstance(message_content, list):
        assistant_message = {
            "role": "assistant",
            "content": message_content
        }
    elif hasattr(message_content, "content"):
        content_list = []
        for block in message_content.content:
            if block.type == "text":
                content_list.append({"type": "text", "text": block.text})
            elif block.type == "tool_use":
                content_list.append({
                    "type": "tool_use",
                    "id": block.id,
                    "name": block.name,
                    "input": block.input
                })
        assistant_message = {
            "role": "assistant",
            "content": content_list
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": [{"type": "text", "text": message_content}]
        }
    
    logger.info(f"Adding assistant message: {assistant_message}")

    messages.append(assistant_message)

def chat(messages, model=MODEL, temperature=TEMPERATURE, system=None, stop_sequences=None, tools=None, tool_choice=None, betas=[]):
    try:
        params = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": 1000,
            "stop_sequences": stop_sequences,
        }
        if system:
            params["system"] = system

        if tools:
            params["tools"] = tools

        if tool_choice:
            params["tool_choice"] = tool_choice

        if betas:
            params["betas"] = betas

        if stop_sequences:
            params["stop_sequences"] = stop_sequences

        # Use client.beta.messages.stream for streaming
        return client.messages.create(**params)
        
    except Exception as e:
        logger.error(f"Chat streaming failed: {e}")
        raise e
    
def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

# tool schema

In [4]:
web_search_schema = {
    "type": "web_search_20250305",
    "name": "web_search",
    "max_uses": 5,
    "allowed_domains": ["nih.gov"],
}

# main function

In [7]:
def main(prompt):
    messages = []
    add_user_message(
        messages,
        prompt,
    )
    
    # Print the user message
    print("=" * 50)
    print("USER:")
    print("=" * 50)
    print(prompt)
    print()
    
    response = chat(messages, tools=[web_search_schema])

    logger.info(f"Response: {response}")
    
    # Print the assistant message
    print("=" * 50)
    print("ASSISTANT:")
    print("=" * 50)
    print(text_from_message(response))
    print()
    
    return response

# test case

In [8]:
prompt = """What's the best exercise for gaining back muscle?"""
main(prompt)

USER:
What's the best exercise for gaining back muscle?



2025-08-08 13:58:50 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-08-08 13:58:51 - __main__ - INFO - Response: Message(id='msg_018LYFRuZhWWT7UuQYEgvz6V', content=[TextBlock(citations=None, text='Let me search for expert recommendations on the best exercises for building back muscles.', type='text'), ServerToolUseBlock(id='srvtoolu_01YL7bK2EWDBND2F66btNHLJ', input={'query': 'most effective exercises for back muscle growth scientific research'}, name='web_search', type='server_tool_use'), WebSearchToolResultBlock(content=[WebSearchResultBlock(encrypted_content='EtYeCioIBhgCIiQxOTM0MGQ5YS1iOTk1LTQyNGEtODY5Mi1kMDBiZTljYmY5MDISDNtgdeafwwjwSE/tvhoMxQhazwKjCCs2f/ubIjAtznLixkJPyv4scYbC/ZlrrobqNzBzrxwEagbdPnWh2o7P71Ft3cGjU+fgEjpJi6Uq2R11f/qyVFFHAR1syjPJ4qWMXVz5ca8eG0aUCXjWmZEXzQk+wnUImv7bKHz3Jf0DSb36lTVzMAIVWVwSW6c891Ryuua6c7FVcOvkpJpRxlw0Hbb1N2EPQOV96yZgbPMWwYrKSsrRXf5SDN5d93n6lVsEjnc6KXFSrTe8b8RycXO44JM+7JZg+04cgLvUqg6fnxXIeY7Lt7ha8KiYDC+boBy

ASSISTANT:
Let me search for expert recommendations on the best exercises for building back muscles.
Based on the search results, I'll provide a comprehensive answer about the most effective exercises for building back muscles:

1. Training Principles:

Resistance training is a primary method for developing strength and muscle growth. Research shows you can achieve significant muscle growth when:
- Training with moderate to high loads
- Keeping about 3-4 repetitions in reserve in most sets
- Focusing on total training volume (sets × repetitions)


2. Key Exercise Components:

An effective back training program should incorporate:
- Muscular strength training
- Flexibility work
- Core strength development to support the lumbar spine
- Some aerobic elements to improve blood flow and nutrient delivery to back muscles


3. Training Guidelines:

For optimal results:
- Train back muscles at least twice per week
- Allow recovery time between sessions by not working the same muscle group on co

Message(id='msg_018LYFRuZhWWT7UuQYEgvz6V', content=[TextBlock(citations=None, text='Let me search for expert recommendations on the best exercises for building back muscles.', type='text'), ServerToolUseBlock(id='srvtoolu_01YL7bK2EWDBND2F66btNHLJ', input={'query': 'most effective exercises for back muscle growth scientific research'}, name='web_search', type='server_tool_use'), WebSearchToolResultBlock(content=[WebSearchResultBlock(encrypted_content='EtYeCioIBhgCIiQxOTM0MGQ5YS1iOTk1LTQyNGEtODY5Mi1kMDBiZTljYmY5MDISDNtgdeafwwjwSE/tvhoMxQhazwKjCCs2f/ubIjAtznLixkJPyv4scYbC/ZlrrobqNzBzrxwEagbdPnWh2o7P71Ft3cGjU+fgEjpJi6Uq2R11f/qyVFFHAR1syjPJ4qWMXVz5ca8eG0aUCXjWmZEXzQk+wnUImv7bKHz3Jf0DSb36lTVzMAIVWVwSW6c891Ryuua6c7FVcOvkpJpRxlw0Hbb1N2EPQOV96yZgbPMWwYrKSsrRXf5SDN5d93n6lVsEjnc6KXFSrTe8b8RycXO44JM+7JZg+04cgLvUqg6fnxXIeY7Lt7ha8KiYDC+boBye8HlY/+j4/vi54K1SnxkAExWOb1gB55Z0SBMQ5JrCYLWGjhoWIUchmrQ0fikrgaaYnryUL0aKXl18OdvkikZWvHkWyLSsc2eFjt/ciwz10S5tScfG73wduyWDYJx1677d/K+3G7xCeDTAcAGI/zcevfkRMQdkE5Oyj